<a href="https://colab.research.google.com/github/CHOISEONGGU/DeepLearning/blob/main/CHOISEONGGU/%EA%B3%B5%EB%B6%80%EC%9E%90%EB%A3%8C/ch09_DL_12_LSTM_ChoiSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# https://www.kaggle.com/{본인ID}/account
# 본인 프로필 누르고 > Account
# > Create New API Token > kaggle.json
from google.colab import files

!pip install -q kaggle #kaggle 설치 # --quiet
# -- kaggle api를 사용하기 위한 인증파일을 설정 (8-11)
files.upload() #kaggle API file upload
!mkdir ~/.kaggle # kaggle 디렉토리 생성 / mkdir : make directory - 폴더 생성 (~)
!cp kaggle.json ~/.kaggle/ #kaggle.json 파일 kaggle 폴더에 복사 / cp a b (copy)
!chmod 600 ~/.kaggle/kaggle.json # 권한 변경 r w x (4 2 1)
# ---------
# !kaggle datasets list #kaggle 데이터셋 리스트 체크
# https://www.kaggle.com/datasets/aashita/nyt-comments
# ... > Copy API command > 예) kaggle datasets download -d aashita/nyt-comments
!kaggle datasets download -d aashita/nyt-comments
!ls # 다운받은 파일 리스트 확인 -> nyt-comments.zip

y
y
y
y


Saving kaggle.json to kaggle (2).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
nyt-comments.zip: Skipping, found more recently modified local copy (use --force to force download)
 ArticlesApril2017.csv	 ArticlesMay2017.csv	 CommentsMarch2018.csv
 ArticlesApril2018.csv	 CommentsApril2017.csv	 CommentsMay2017.csv
 ArticlesFeb2017.csv	 CommentsApril2018.csv	'kaggle (1).json'
 ArticlesFeb2018.csv	 CommentsFeb2017.csv	'kaggle (2).json'
 ArticlesJan2017.csv	 CommentsFeb2018.csv	 kaggle.json
 ArticlesJan2018.csv	 CommentsJan2017.csv	 nyt-comments.zip
 ArticlesMarch2017.csv	 CommentsJan2018.csv	 sample_data
 ArticlesMarch2018.csv	 CommentsMarch2017.csv


In [61]:
!unzip nyt-comments

Archive:  nyt-comments.zip
replace ArticlesApril2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesApril2017.csv   
replace ArticlesApril2018.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesApril2018.csv   
replace ArticlesFeb2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesFeb2017.csv     
replace ArticlesFeb2018.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesFeb2018.csv     
replace ArticlesJan2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesJan2017.csv     
replace ArticlesJan2018.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesJan2018.csv     
replace ArticlesMarch2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesMarch2017.csv   
replace ArticlesMarch2018.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesMarch2018.csv   
replace ArticlesMay2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ArticlesMay2017.csv     
re

> colab ubuntu에서 tar 포맷을 압축 푸는 방법을 알려줘.
linux 기능을 활용한 방법과, 파이썬의 패키지를 사용하는 방법으로.

> colab ubuntu에서 zip 포맷을 압축 푸는 방법을 알려줘.
linux 기능을 활용한 방법과, 파이썬의 패키지를 사용하는 방법으로.

* https://sharegpt.com/c/1khtmNX

In [62]:
import pandas as pd

df = pd.read_csv('ArticlesApril2017.csv')

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   abstract          22 non-null     object
 1   articleID         886 non-null    object
 2   articleWordCount  886 non-null    int64 
 3   byline            886 non-null    object
 4   documentType      886 non-null    object
 5   headline          886 non-null    object
 6   keywords          886 non-null    object
 7   multimedia        886 non-null    int64 
 8   newDesk           886 non-null    object
 9   printPage         886 non-null    int64 
 10  pubDate           886 non-null    object
 11  sectionName       886 non-null    object
 12  snippet           886 non-null    object
 13  source            886 non-null    object
 14  typeOfMaterial    886 non-null    object
 15  webURL            886 non-null    object
dtypes: int64(3), object(13)
memory usage: 110.9+ KB


In [64]:
df.columns # 사람이 작성한 자연어 데이터 'headline'만 사용

Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [65]:
df.headline

0      Finding an Expansive View  of a Forgotten Peop...
1                      And Now,  the Dreaded Trump Curse
2                  Venezuela’s Descent Into Dictatorship
3                  Stain Permeates Basketball Blue Blood
4                              Taking Things for Granted
                             ...                        
881                  Reporting on Gays Who ‘Don’t Exist’
882    The Fights That Could Lead to a Government Shu...
883    ‘The Leftovers’ Season 3, Episode 2: Swedish P...
884                          Thinking Out Loud, But Why?
885                    Some Sugar. Could Use More Spice.
Name: headline, Length: 886, dtype: object

# 데이터 전처리

1. 특수문자 제거
1. BOW (Bag of words - 모든 단어를 겹치지 않도록 고유한 번호로 나타낸 집합 = 단어 사전)

In [66]:
import numpy as np
import glob # 파일을 패턴화해서 읽을 수 있게 도와주는 라이브러리

from torch.utils.data.dataset import Dataset
import string # 특수문제 제거용 (punctuation)

In [67]:
# 특수문자 제거용 함수 정의
# -> 모든 문자를 소문자로 바꾸고 특수문자를 제거
def clean_text(txt):
    new_text = []
    for v in txt: # 문자 한 개씩 반환
        if v not in string.punctuation: # 특수문자가 아닌 것만 남기기
            new_text.append(v)
    return "".join(new_text).lower() # 소문자화
    # https://dojang.io/mod/page/view.php?id=2305
    # gitbook :  https://prorice.gitbooks.io/gitbook/content/
    # Confluence : https://blog.naver.com/PostView.naver?blogId=interj77&logNo=221761881031
# 이 코드를 삼항연산자와 리스트 컴프리헨션을 쓴 코드로 바꿔주고, 두 코드의 차이점을 주석으로 나타내줘
# https://shareg.pt/hgszAKX
# https://tes-b.github.io/etc/vscode_with_gpt/
# https://namu.wiki/w/GitHub%20Copilot

In [68]:
clean_text('Hello, Hello? hELLO!!!')

'hello hello hello'

In [69]:
# https://m.blog.naver.com/siniphia/221397012627
glob.glob("*.csv") # 특정한 확장자를 가진 모든 파일 검색 

['ArticlesMay2017.csv',
 'ArticlesFeb2017.csv',
 'ArticlesFeb2018.csv',
 'ArticlesApril2018.csv',
 'ArticlesApril2017.csv',
 'ArticlesJan2017.csv',
 'ArticlesJan2018.csv',
 'CommentsApril2018.csv',
 'CommentsApril2017.csv',
 'CommentsMay2017.csv',
 'CommentsFeb2017.csv',
 'CommentsFeb2018.csv',
 'ArticlesMarch2017.csv',
 'CommentsMarch2017.csv',
 'ArticlesMarch2018.csv',
 'CommentsJan2017.csv',
 'CommentsJan2018.csv',
 'CommentsMarch2018.csv']

In [70]:
glob.glob("Articles*.csv") # 특정한 확장자와 접두사를 가진 모든 파일 검색

['ArticlesMay2017.csv',
 'ArticlesFeb2017.csv',
 'ArticlesFeb2018.csv',
 'ArticlesApril2018.csv',
 'ArticlesApril2017.csv',
 'ArticlesJan2017.csv',
 'ArticlesJan2018.csv',
 'ArticlesMarch2017.csv',
 'ArticlesMarch2018.csv']

In [71]:
glob.glob("*2017.csv") # 특정한 확장자와 접미사를 가진 모든 파일 검색

['ArticlesMay2017.csv',
 'ArticlesFeb2017.csv',
 'ArticlesApril2017.csv',
 'ArticlesJan2017.csv',
 'CommentsApril2017.csv',
 'CommentsMay2017.csv',
 'CommentsFeb2017.csv',
 'ArticlesMarch2017.csv',
 'CommentsMarch2017.csv',
 'CommentsJan2017.csv']

In [72]:
glob.glob("Articles*2017.csv")

['ArticlesMay2017.csv',
 'ArticlesFeb2017.csv',
 'ArticlesApril2017.csv',
 'ArticlesJan2017.csv',
 'ArticlesMarch2017.csv']

In [73]:
glob.glob("*May*.csv")

['ArticlesMay2017.csv', 'CommentsMay2017.csv']

> 이런 파일들이 있다고 했을 때,
2018로 끝나는 csv 파일만 검색하는 glob 검색식이 어떻게 돼?
* https://sharegpt.com/c/lfX8mlY

* https://www.deepl.com/ko/translator
* 크롬 익스텐션 깔고 -> ctrl + shift + y


# 데이터셋

In [74]:
all_headlines = []
# Articles로 시작하는 csv파일 이름을 모두 검색해서 가져오는 코드
for filename in glob.glob("Articles*.csv"):
    article_df = pd.read_csv(filename) # 파일이름으로 csv를 df로 만들어줌
    # df.headline : headline이라는 이름의 컬럼(열)만 불러오기
    # -> index, value 시리즈 -> values -> 인덱스없이 '값'만 불러온 ndarray
    # -> list (  ) -> 리스트 타입으로 바뀜 -> all_headlines와 연결
    all_headlines.extend(list(article_df.headline.values))
    # all_headlines += list(article_df.headline.values)

In [75]:
# [pd.read_csv(filename) for filename in glob.glob("Articles*.csv")]
# [pd.read_csv(filename)['headline'] for filename in glob.glob("Articles*.csv")]
all_headlines_df = pd.concat(
    [pd.read_csv(filename)['headline'] for filename in glob.glob("Articles*.csv")],
    ignore_index=True,)

In [76]:
all_headlines

['My Beijing: The Sacred City',
 '6 Million Riders a Day, 1930s Technology',
 'Seeking a Cross-Border Conference',
 'Questions for: ‘Despite the “Yuck Factor,” Leeches Are Big in Russian Medicine’',
 'Who Is a ‘Criminal’?',
 'An Antidote to Europe’s Populism',
 'The Cost of a Speech',
 'Degradation of the Language',
 'On the Power of Being Awful',
 'Trump Garbles Pitch on a Revised Health Bill',
 'What’s Going On in This Picture? | May 1, 2017',
 'Unknown',
 'When Patients Hit a Medical Wall',
 'Unknown',
 'For Pregnant Women, Getting Serious About Whooping Cough',
 'Unknown',
 'New York City Transit Reporter in Wonderland: Riding the London Tube',
 'How to Cut an Avocado Without Cutting Yourself',
 'In Fictional Suicide, Health Experts Say They See a Real Cause for Alarm',
 'Claims of Liberal Media Bias Hit ESPN, Too',
 'Is the dream in Australia crumbling?',
 'Police in Texas Change Account in Officer’s Fatal Shooting of 15-Year-Old',
 'Most Adults Favor Sex Ed. Most Students Don’t G

In [77]:
all_headlines_df.value_counts()

Unknown                                                                           732
Variety: Acrostic                                                                  16
A Little Variety                                                                   10
Homes for Sale in Manhattan and Brooklyn                                            7
Variety Puzzle: Acrostic                                                            6
                                                                                 ... 
Hot Stuff Turns Cold                                                                1
A Quote Disproved                                                                   1
Choosing Brexit, a Town Yearned for Its Seafaring Past, and Muddied Its Future      1
My Life-Threatening Bout With E. Coli Food Poisoning                                1
Can a Co-op Message Board Function as an Open Forum?                                1
Name: headline, Length: 8310, dtype: int64

In [78]:
all_headlines_df = all_headlines_df[all_headlines_df != 'Unknown']
all_headlines_df

0                             My Beijing: The Sacred City
1                6 Million Riders a Day, 1930s Technology
2                       Seeking a Cross-Border Conference
3       Questions for: ‘Despite the “Yuck Factor,” Lee...
4                                    Who Is a ‘Criminal’?
                              ...                        
9330                      Will Trump Start a Farm Crisis?
9331                           A New Black American Dream
9332                       When a Subject Refuses to Pose
9333                   America Needs Better Privacy Rules
9334                                  Rhythms  Of Tragedy
Name: headline, Length: 8603, dtype: object

In [79]:
all_headlines_df.reset_index(inplace=True, drop=True)
all_headlines_df

0                             My Beijing: The Sacred City
1                6 Million Riders a Day, 1930s Technology
2                       Seeking a Cross-Border Conference
3       Questions for: ‘Despite the “Yuck Factor,” Lee...
4                                    Who Is a ‘Criminal’?
                              ...                        
8598                      Will Trump Start a Farm Crisis?
8599                           A New Black American Dream
8600                       When a Subject Refuses to Pose
8601                   America Needs Better Privacy Rules
8602                                  Rhythms  Of Tragedy
Name: headline, Length: 8603, dtype: object

In [80]:
all_headlines_df.apply(clean_text)

0                              my beijing the sacred city
1                 6 million riders a day 1930s technology
2                        seeking a crossborder conference
3       questions for ‘despite the “yuck factor” leech...
4                                     who is a ‘criminal’
                              ...                        
8598                       will trump start a farm crisis
8599                           a new black american dream
8600                       when a subject refuses to pose
8601                   america needs better privacy rules
8602                                  rhythms  of tragedy
Name: headline, Length: 8603, dtype: object

In [81]:
all_headlines = [h for h in all_headlines if h != 'Unknown']

In [82]:
all_headlines

['My Beijing: The Sacred City',
 '6 Million Riders a Day, 1930s Technology',
 'Seeking a Cross-Border Conference',
 'Questions for: ‘Despite the “Yuck Factor,” Leeches Are Big in Russian Medicine’',
 'Who Is a ‘Criminal’?',
 'An Antidote to Europe’s Populism',
 'The Cost of a Speech',
 'Degradation of the Language',
 'On the Power of Being Awful',
 'Trump Garbles Pitch on a Revised Health Bill',
 'What’s Going On in This Picture? | May 1, 2017',
 'When Patients Hit a Medical Wall',
 'For Pregnant Women, Getting Serious About Whooping Cough',
 'New York City Transit Reporter in Wonderland: Riding the London Tube',
 'How to Cut an Avocado Without Cutting Yourself',
 'In Fictional Suicide, Health Experts Say They See a Real Cause for Alarm',
 'Claims of Liberal Media Bias Hit ESPN, Too',
 'Is the dream in Australia crumbling?',
 'Police in Texas Change Account in Officer’s Fatal Shooting of 15-Year-Old',
 'Most Adults Favor Sex Ed. Most Students Don’t Get It.',
 'Australia Feels Its Ties 

> LSTM을 통한 자연어 분석을 하려고 하는데, torchtext 사용하지 않고, 텍스트의 BoW 또는 단어사전을 만드는 방법이 뭐가 있어? 딕셔너리를 사용하거나, 외부 라이브러리를 사용하는 방법을 3개 제시해줘
* https://sharegpt.com/c/slOXE7n

In [83]:
# PyTorch에서 쓰일 자연어 데이터셋 (torchtext 안 쓰는 버전)
class TextGeneration(Dataset): # 상속 받아주는 class -> Dataset -> DataLoader 때문이다.
    def clean_text(self, txt):
        # new_text = []
        # for v in txt: # 문자 한 개씩 반환
        #     if v not in string.punctuation: # 특수문자가 아닌 것만 남기기
        #         new_text.append(v)
        # return "".join(new_text).lower() # 소문자화
        return "".join([v for v in txt if v not in string.punctuation]).lower()
    # 생성자
    def __init__(self) -> None:
        super().__init__()
        # headline 사용 (나머지는 메타데이터라 의미 X)
        # Article만 사용
        # all_headlines = []
        # # Articles로 시작하는 csv파일 이름을 모두 검색해서 가져오는 코드
        # for filename in glob.glob("Articles*.csv"):
        #     article_df = pd.read_csv(filename) # 파일이름으로 csv를 df로 만들어줌
        #     # df.headline : headline이라는 이름의 컬럼(열)만 불러오기
        #     # -> index, value 시리즈 -> values -> 인덱스없이 '값'만 불러온 ndarray
        #     # -> list (  ) -> 리스트 타입으로 바뀜 -> all_headlines와 연결
        #     all_headlines.extend(list(article_df.headline.values))
        #     # all_headlines += list(article_df.headline.values)
        # # headline 중 unknown을 제거 (필터링)
        # all_headlines = [h for h in all_headlines if h != 'Unknown']
        all_headlines = pd.concat(
            [pd.read_csv(fn)['headline']
            for fn in glob.glob("Articles*.csv")],
            ignore_index=True).loc[lambda x:x != 'Unknown'].reset_index(drop=True)
        # all_headlines = all_headlines[all_headlines != 'Unknown'].reset_index(drop=True)
        # 특수문자 등 텍스트 전처리
        # self.corpus = [self.clean_text(x) for x in all_headlines]
        self.corpus = all_headlines.apply(self.clean_text).values 
        # vlaues -> pandas Series를 numpy배열로 바꿔서 for문 처리를 효율적으로 처리
        
        # corpus : 말뭉치
        # BOW : Bag of Words => 등장하는 모든 단어에 고유번호 지정
        self.BOW = {} # 단어사전전
        for line in self.corpus: # corpus에서 headline에서 추출한 문장들을 불러오기
            for word in line.split(): # 띄어쓰기를 기준으로 토큰화 -> 단어
                if word not in self.BOW.keys(): # 단어가 BOW에 있니?
                    self.BOW[word] = len(self.BOW.keys()) # 고유번호를 지정
        # 모델의 입력으로 사용할 데이터(self.data)
        self.data = self.generate_sequence(self.corpus)
        # sequence -> (1번, 2번) => (3번)
        # 단어 2개를 넣어서 3번째 단어를 유추

    # 텍스트 시계열 -> BOW
    # (단어1, 단어2) => (단어3)
    def generate_sequence(self, txt):
        seq = [] #([a, b], c)
        for line in txt: # 전처리가 된 self.corpus 데이터 = 헤드라인
            # 1단계
            line = line.split() # 토큰화 (스페이스 기준으로)
            # BOW(사전) -> key -> 고유번호 지정
            # apple -> 1, is -> 2, delicious -> 3
            line_bow = [self.BOW[word] for word in line] # 1, 2, 3 -> 문장을 고유번호화
            # line 행들 -> bow를 통해서 고유번호화 된 문장 (한 행들)
            # 단어 2개를 입력 -> 그 다음 단어가 정답(예측 대상)
            # data = []
            ## for i in range(len(line_bow)-2): # -2가 있는 이유 ?=> 3개를 학습해야하는데, 뒤에서 2번째부터는 본인 포함해서 2개밖에 학습데이터가 안됌, 그래서 애초에 인덱스로 컷해준다.
            ## line bow -> len => line에 있는 스페이스를 기준으로 split(토큰화 : 단어기반으로 쪼개진)된 단어들을 가진 리스트 -> bow를 통해서 단어들이 고유번호로 대체된 상태
            ## len -> 특정 고유번호로 대체된 리스트의 길이이다. 여기에 -2를 해준다는 => 10개의 단어 => [8] 인덱스 ( 본인, 본인 +1 )이 있는데 10이 없다??
            #     # 0 -> 0, 1 => 2 / 1 -> 1, 2 => 3 / ... / n-3 -> n-2, n-1 => n-1
            #     # d = ([a, b], c)
            #     d = ([line_bow[i], line_bow[i+1]], line_bow[i+2]) # a, b를 사용해서, c를 예측 (순서대로)
            #     data.append(d)
            data = [([line_bow[i], line_bow[i+1]], line_bow[i+2])
                    for i in range(len(line_bow)-2)]
            # seq.extend(data)
            seq += data
        return seq
    
    # 데이터 개수를 반환하는 함수
    def __len__(self):
        return len(self.data)
    
    # 데이터를 불러오는 함수
    def __getitem__(self, i):
        data = np.array(self.data[i][0]) # (1번, 2번 단어)
        label = np.array(self.data[i][1]) # (3번 단어)
        return data, label

In [84]:
import torch.nn as nn
import torch

# 모델 정의

* https://scholar.google.com/
* http://www.riss.kr/index.do
* https://dl.nanet.go.kr/#none

In [90]:
class LSTM(nn.Module):
    def __init__(self, num_embeddings) -> None:
        super().__init__()
        # 임베딩층 구성
        # 단어 임베딩에서 밀집 표현과 희소 표현의 차이를 알려줘 (딥러닝 임베딩, 원핫인코딩, 라벨 인코딩 등의 표현을 포함해서)
        # https://shareg.pt/yYCNApF

        # [임베딩층]
        # I love pie
        # [0, 0, 0, 0, 0, 1...] # 희소표현
        # 자연어 처리 -> 단어 개수 -> 엄청 많다
        # 모델에 들어가는 대부분의 입력값이 0이 됨
        # => 희소표현 (학습 제대로 X) => 가중치 곱해도 0
        # 희소표현 => 밀집표현
        # 밀집표현을 위한 임베딩 층
        # num_embeddings => num_vocab => bow의 고유단어 갯수
        # data : ? , 2, 단어 갯수
        self.embed = nn.Embedding(
            num_embeddings = num_embeddings,
            embedding_dim=16,
        )
        # embedding : ?, 2, 16

        # LSTM 5개 층을 쌓음
        self.lstm = nn.LSTM(
            input_size = 16, 
            hidden_size = 64,
            num_layers = 5,
            batch_first = True,
        )
        # batch, sequence, feature
        #  (?) ,    2    ,   64
        
        # 분류 (fc)
        self.fc1 = nn.Linear(2 * 64, num_embeddings) # => 다시 고유단어 갯수로 분류
        self.fc2 = nn.Linear(num_embeddings, num_embeddings)
        # 정확도를 높이기 위한 분류기 한 더 사용

        # 활성화 함수
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    
    # 순전파
    def forward(self, x):
        # bow 고유단어 갯수 (2, ...) -> 16 (2, 16)
        x = self.embed(x) # 희소 -> 밀집
        # LSTM 모델의 예측값
        x, _ = self.lstm(x) # 전체 출력 상태 + 셀 상태값
        # (2, 16) -> (2, 64)
        x = torch.reshape(x, (x.shape[0], -1)) # batch, 2 * 64 => batch, 128
        # x = torch.reshape(x, (-1, 128)) # batch, 2 * 64 => batch, 128
        # 분류기
        # 128 -> 단어 수
        x = self.fc1(x) # 128개 가 -> 단어 수로 가게된다.
        x = self.tanh(x)
        x = self.fc2(x) # 단어수 -> 단어수
        # return
        return x

# 학습

In [92]:
from tqdm.notebook import tqdm # 진척도

from torch.utils.data import DataLoader # 배치사이즈로 끊어서 데이터 제공
from torch.optim.adam import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [93]:
dataset = TextGeneration()

In [94]:
loader = DataLoader(dataset, batch_size=64)

In [95]:
model = LSTM(num_embeddings=len(dataset.BOW)).to(device)
optim = Adam(model.parameters(), lr=0.001) # 0.001 ~ 0.00001

In [96]:
# for epoch in range(200):
for epoch in range(50):
    iterator = tqdm(loader)
    # data: 입력값 (1번째, 2번째 단어) / label : 정답값 (3번째 단어)
    for data, label in iterator:
        optim.zero_grad() # 기울기 초기화
        # 모델의 예측값
        # pred = model(torch.tensor(data, dtype=torch.long).to(device))
        data = torch.tensor(data, dtype=torch.long).to(device)
        pred = model(data)
        # 정답 레이블
        label = torch.tensor(label, dtype=torch.long).to(device)
        # 오차
        loss = nn.CrossEntropyLoss()(pred, label)
        # 오차 역전파
        loss.backward()
        optim.step()

        iterator.set_description(f"epoch{epoch} / loss: {loss.item()}")

torch.save(model.state_dict(), "lstm.pt")

  0%|          | 0/677 [00:00<?, ?it/s]

<ipython-input-96-4f76160751c0>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.long).to(device)
<ipython-input-96-4f76160751c0>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.long).to(device)


  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

...

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

In [98]:
# 문장 만들기 -> 2개 이상의 단어로 구성된 문장을 넣으면 => 다음 단어를 유추
def generate(model, BOW, string="finding an ", strlen=10):
    print(f"input word : {string}")

    with torch.no_grad(): # 기울기 계산 끄고,
        for p in range(strlen):
            # 입력한 문장을 텐서화
            # [BOW[w] for w in string.split()] -> string을 스페이스 단위로 분할, 해당 토큰(단어)들이 BOW에서 어디에 있는지 확인
            # torch.tensor(..., dtype=torch.long).to(device) -> 모델이 이해할 수 있는 텐서로 변환
            words = torch.tensor([BOW[w] for w in string.split()], dtype=torch.long).to(device)
            # 모델의 입력으로 입력으로 사용하기 위한 배치 차원을 추가
            input_tensor = torch.unsqueeze(words[-2:], dim=0) # 마지막 두 단어만 추출해서 -> 두 단어를 배치 차원 반영하여 분석
            output = model(input_tensor) # 모델을 통해 예측
            # torch.argmax(output) -> 가장 확률값이 높은 위치의 텐서의 인덱스
            # 0~전체사전의 길이만큼의 텐서. 확률값 -> 단어의 고유번호
            # (...).cpu().numpy() -> (cuda-gpu) => cpu => numpy 배열
            output_word = (torch.argmax(output).cpu().numpy())
            string += list(BOW.keys())[output_word]
            string += " "
        
    print(f"예측 문장 : {string}")

In [ ]:
model.load_state_dict(torch.load("lstm.pt", map_location=device))
pred = generate(model, dataset.BOW)

input word : finding an 
예측 문장 : finding an wildfires deserved in woo at ms woman’ made woman’ options 


In [ ]:
pred = generate(model, dataset.BOW, 'red apple ')

input word : red apple 
예측 문장 : red apple wells woman’ jianbing spread coldest grants ct insured is in 


In [ ]:
pred = generate(model, dataset.BOW, 'blue sky ')

input word : blue sky 
예측 문장 : blue sky tragedy in sit of the true commute of budget’ unclear 


In [ ]:
pred = generate(model, dataset.BOW, 'smart computer ')

input word : smart computer 
예측 문장 : smart computer tragedy in sit of the true commute of budget’ unclear 


In [99]:
pred = generate(model, dataset.BOW, 'hello world ')

input word : hello world 
예측 문장 : hello world doctor’ again to know to privacy arabia visions jitters 2018 


In [100]:
pred = generate(model, dataset.BOW, 'hello world ', 20)

input word : hello world 
예측 문장 : hello world doctor’ again to know to privacy arabia visions jitters 2018 exbodega sand capital hidden waning men’s abused visions of setbacks 


* https://wikidocs.net/45101